In [36]:
import numpy as np
import pandas as pd
from scipy.stats import norm

import scipy
from statsmodels.stats.weightstats import *
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

## Q1

Question 1

Пусть t  — значение статистики критерия Стьюдента для независимых выборок, nu  —  соответствующее число степеней свободы. Какая из строчек кода вычисляет достигаемый уровень значимости t-критерия при альтернативе H1 ⁣:μ1>μ2H_1\colon \mu_1>\mu_2H1​:μ1​>μ2​?

In [62]:
t= 2.27
nu = 3
1-stats.t.cdf(t, nu)

0.05396254003922896

## Question 4

Уровень кальция в крови здоровых молодых женщин равен в среднем 9.5 милиграммам на децилитр и имеет характерное стандартное отклонение 0.4 мг/дл. В сельской больнице Гватемалы для 160 здоровых беременных женщин при первом обращении для ведения беременности был измерен уровень кальция; среднее значение составило 9.57 мг/дл. Можно ли утверждать, что средний уровень кальция в этой популяции отличается от 9.5?

Посчитайте достигаемый уровень значимости. Поскольку известны только среднее и дисперсия, а не сама выборка, нельзя использовать стандартные функции критериев — нужно реализовать формулу достигаемого уровня значимости самостоятельно.

Округлите ответ до четырёх знаков после десятичной точки.

In [2]:
x_mean = 9.57 
true_mean = 9.5
true_std = 0.4
n = 160

In [3]:
z = (x_mean - true_mean)/(true_std/np.sqrt(n))
z

2.213594362117875

In [7]:
norm.ppf(0.95)

1.6448536269514722

Одностронняя альтернатива

In [22]:
1 - norm.cdf(z)

0.013428347753761893

Двустороняя альтернатива

In [63]:
round((2 * (1 - stats.norm.cdf(z))),4)

0.0269

In [14]:
norm.cdf(norm.ppf(0.95))

0.95

Доверительный интервал для среднего

In [58]:
stats.norm.interval(0.95, loc=x_mean, scale=(true_std/np.sqrt(n)))

(9.508020496769545, 9.631979503230456)

In [59]:
#stats.norm.interval(0.95, loc=x_mean, scale=true_std)

(8.786014406183979, 10.353985593816022)

## Question 6

Отделите 25% случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). На обучающей выборке настройте две регрессионные модели:

    линейную регрессию с помощью LinearRegression без параметров

    случайный лес из 10 деревьев с помощью RandomForestRegressor с random_state=1.

Какая из моделей лучше предсказывает цену бриллиантов? Сделайте предсказания на тестовой выборке, посчитайте модули отклонений предсказаний от истинных цен. Проверьте гипотезу об одинаковом среднем качестве предсказаний, вычислите достигаемый уровень значимости. Отвергается ли гипотеза об одинаковом качестве моделей против двусторонней альтернативы на уровне значимости α=0.05\alpha=0.05α=0.05?

In [27]:
df = pd.read_csv('diamonds.txt', sep="\t")
df

,carat,depth,table,price,x,y,z
0,0.23,61.5,55.0,326,3.95,3.98,2.43
1,0.21,59.8,61.0,326,3.89,3.84,2.31
2,0.23,56.9,65.0,327,4.05,4.07,2.31
3,0.29,62.4,58.0,334,4.20,4.23,2.63
4,0.31,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...
53935,0.72,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,61.0,58.0,2757,6.15,6.12,3.74


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   carat   53940 non-null  float64
 1   depth   53940 non-null  float64
 2   table   53940 non-null  float64
 3   price   53940 non-null  int64  
 4   x       53940 non-null  float64
 5   y       53940 non-null  float64
 6   z       53940 non-null  float64
dtypes: float64(6), int64(1)
memory usage: 2.9 MB


In [31]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('price', axis=1), df['price'], test_size=0.25, random_state=1)

In [33]:
LR = LinearRegression().fit(X_train, y_train)
lr_pred = LR.predict(X_test)

In [73]:
RF = RandomForestRegressor(n_estimators=10, random_state=1).fit(X_train, y_train)
rf_pred = RF.predict(X_test)

In [74]:
lr_mae = mean_absolute_error(y_test, lr_pred)
rf_mae = mean_absolute_error(y_test, rf_pred)
lr_mae, rf_mae

(890.3764004285612, 802.9205172724115)

In [75]:
stats.ttest_rel(abs(y_test - lr_pred), 
                abs(y_test - rf_pred))

Ttest_relResult(statistic=13.017729783878707, pvalue=1.6551745751382525e-38)

In [76]:
print("95%% confidence interval: [%f, %f]" % DescrStatsW(abs(y_test - lr_pred)-abs(y_test - rf_pred)).tconfint_mean())

95% confidence interval: [74.287245, 100.624521]


## Q7

In [38]:
error_lr = np.abs(lr_pred - y_test)
error_rf = np.abs(rf_pred - y_test)

In [42]:
stats.ttest_rel(error_lr, error_rf)

Ttest_relResult(statistic=-50.5458981725294, pvalue=0.0)

In [54]:
print("95%% confidence interval: [%f, %f]" % DescrStatsW(error_lr - error_rf).tconfint_mean())

95% confidence interval: [-1061.646782, -982.380613]


In [45]:
scipy.stats.ttest_ind(error_lr,error_rf, equal_var = False)

Ttest_indResult(statistic=-55.443758391236024, pvalue=0.0)

In [47]:
(lr_pred - y_test).describe()

count    13485.000000
mean        19.229235
std       1463.058136
min     -12455.940789
25%       -342.670547
50%         63.649682
75%        652.518106
max      18239.846360
Name: price, dtype: float64

In [77]:
print("95%% confidence interval: [%f, %f]" % DescrStatsW(np.abs(y_test - rf_pred) - np.abs(y_test- lr_pred)).tconfint_mean())

95% confidence interval: [-100.624521, -74.287245]


In [72]:
error_rf.describe()

count    13485.000000
mean      1041.242933
std       1562.530902
min          0.000000
25%        145.000000
50%        400.000000
75%       1186.000000
max      12765.000000
Name: price, dtype: float64